# Recreación del codigo de AffWild2

# TEXTO

In [1]:
import os
from glob import glob
import numpy as np
from tqdm import tqdm
import torch
from speechbrain.pretrained import EncoderDecoderASR
import librosa
import pandas as pd

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


## Construcción de Dataset
Es identico al de audio, pero además se obtiene el texto, agregando este campo al csv.

In [2]:
model = EncoderDecoderASR.from_hparams(
    source="speechbrain/asr-transformer-transformerlm-librispeech",
    savedir="pretrained_models/asr-transformer-transformerlm-librispeech",
#    run_opts={'device':'cuda'},
)
sample_rate = 22400
max_time = 7.

texts = []

In [3]:
import os
import pandas as pd
import librosa

def obtener_nombres_archivos(ruta):
    archivos = []
    for nombre_archivo in os.listdir(ruta):
        if os.path.isfile(os.path.join(ruta, nombre_archivo)):
            ruta_completa = os.path.join(ruta, nombre_archivo)
            duracion = obtener_duracion_audio(ruta_completa)
            archivos.append({'NombreArchivo': nombre_archivo, 'RutaArchivo': ruta_completa, 'Duracion': duracion})
    df = pd.DataFrame(archivos)
    return df

def obtener_duracion_audio(ruta_archivo):
    audio, sr = librosa.load(ruta_archivo)
    duracion = len(audio) / sr
    return duracion

# Ejemplo de uso
ruta_carpeta = r'D:\sentirobots\Audios para pruebas' 
df = obtener_nombres_archivos(ruta_carpeta)
df


,NombreArchivo,RutaArchivo,Duracion
0,00038.wav,D:\sentirobots\Audios para pruebas\00038.wav,5.500000
1,000451774.wav,D:\sentirobots\Audios para pruebas\000451774.wav,2.184036
2,000452240.wav,D:\sentirobots\Audios para pruebas\000452240.wav,3.288027
3,000505440.wav,D:\sentirobots\Audios para pruebas\000505440.wav,2.760000
4,000513320.wav,D:\sentirobots\Audios para pruebas\000513320.wav,1.776009
...,...,...,...
71,video44.wav,D:\sentirobots\Audios para pruebas\video44.wav,259.530159
72,video51.wav,D:\sentirobots\Audios para pruebas\video51.wav,146.913379
73,video60.wav,D:\sentirobots\Audios para pruebas\video60.wav,63.994875
74,video67.wav,D:\sentirobots\Audios para pruebas\video67.wav,61.649660


# SpeechBrain

In [7]:
traindf = pd.DataFrame(columns=['filename', 'Duration','Text'])
for _, row in tqdm(df.iterrows()):
    audiofile, _ = librosa.load(path=row['RutaArchivo'], sr=22400, duration=np.minimum(row['Duracion'], 7.))
    

    if row['Duracion'] < 7.:
        audiofile = librosa.util.pad_center(data=audiofile, size=22400*7)

    normalized = model.audio_normalizer(torch.tensor(audiofile), 22400)
    rel_length = torch.tensor([1.])
    predicted_words, _ = model.transcribe_batch(
        normalized.unsqueeze(0),
        rel_length,
    )

    normalized = normalized.detach()
    rel_length = rel_length.detach()
    torch.cuda.empty_cache()

    rowdf = pd.DataFrame({
        'filename': row['RutaArchivo'],
        'Duration': row['Duracion'],
        'Text': " ".join(predicted_words),
    }, index=pd.Index([0]))

    traindf = pd.concat([traindf, rowdf], ignore_index=True)

traindf

0it [00:00, ?it/s]C:\Users\Desarrollo\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
1it [00:57, 57.85s/it]C:\Users\Desarrollo\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
2it [02:13, 68.29s/it]C:\Users\Desarrollo\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
3it [04:39, 104.02s/it]C:\Users\Desarrollo\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
4it [06:10,

33it [21:06, 37.16s/it]C:\Users\Desarrollo\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
34it [21:19, 29.98s/it]C:\Users\Desarrollo\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
35it [21:34, 25.67s/it]C:\Users\Desarrollo\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
36it [23:09, 46.50s/it]C:\Users\Desarrollo\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
37it

66it [44:14, 34.93s/it]C:\Users\Desarrollo\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
67it [44:43, 33.25s/it]C:\Users\Desarrollo\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
68it [45:44, 41.62s/it]C:\Users\Desarrollo\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
69it [46:04, 35.00s/it]C:\Users\Desarrollo\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
70it

,filename,Duration,Text
0,D:\sentirobots\Audios para pruebas\00038.wav,5.500000,AND THEN IF YOU'D LIKE I'LL INDULGE YOUR CURIO...
1,D:\sentirobots\Audios para pruebas\000451774.wav,2.184036,I DON'T KNOW BEFORE THIS I WAS
2,D:\sentirobots\Audios para pruebas\000452240.wav,3.288027,I NEED TO GET INTO GRIMAUD'S INTO ONE OF THE V...
3,D:\sentirobots\Audios para pruebas\000505440.wav,2.760000,IF YOU ARE NOT DANCING YOU ARE NOT DANCING IN A
4,D:\sentirobots\Audios para pruebas\000513320.wav,1.776009,YOU'RE RIGHT IN THIRTY EIGHT
...,...,...,...
71,D:\sentirobots\Audios para pruebas\video44.wav,259.530159,HULLO MY NAME IS DANNY AND THIS IS MY EXTREMEL...
72,D:\sentirobots\Audios para pruebas\video51.wav,146.913379,I'M TEMPTED TO SAY SMART CREATIVE PEOPLE HAVE NO
73,D:\sentirobots\Audios para pruebas\video60.wav,63.994875,DO YOU MEAN A LIKING I CAN'T SO HE AND GUISE T...
74,D:\sentirobots\Audios para pruebas\video67.wav,61.649660,THERE'S STEVIE AYE AYE AYE AYE AYE


In [8]:
columna = "Text"
num_vacios = traindf[columna].eq('').sum()
total_registros = traindf[columna].shape[0]
porcentaje_vacios = (num_vacios / total_registros) * 100

print("Número de valores vacíos en la columna {}: {}".format(columna, num_vacios))
print("Total de registros en la columna {}: {}".format(columna, total_registros))
print("Porcentaje de valores vacíos en la columna {}: {:.2f}%".format(columna, porcentaje_vacios))

Número de valores vacíos en la columna Text: 0
Total de registros en la columna Text: 76
Porcentaje de valores vacíos en la columna Text: 0.00%


In [11]:
traindf.to_excel(r'D:\sentirobots\Audios para pruebas\transcripcion-SpeechBrain.xlsx', index=False)
print("Realizado")

Realizado


In [4]:
import speech_recognition as sr
from langdetect import detect
import pandas as pd

def detect_language(audio_file):
    recognizer = sr.Recognizer()

    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)

    try:
        transcript = recognizer.recognize_google(audio)
        language = detect(transcript)
        
        return language
    except sr.UnknownValueError:
        print("No se pudo realizar la transcripción del audio.")
    except sr.RequestError as e:
        print(f"Error en la solicitud de reconocimiento de voz: {e}")

columna_ruta = "RutaArchivo"
df_idioma = df.copy()
df_idioma['IdiomaDetectado'] = df_idioma[columna_ruta].apply(detect_language)
df_idioma

No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del audio.
No se pudo realizar la transcripción del

,NombreArchivo,RutaArchivo,Duracion,IdiomaDetectado
0,00038.wav,D:\sentirobots\Audios para pruebas\00038.wav,5.500000,en
1,000451774.wav,D:\sentirobots\Audios para pruebas\000451774.wav,2.184036,en
2,000452240.wav,D:\sentirobots\Audios para pruebas\000452240.wav,3.288027,en
3,000505440.wav,D:\sentirobots\Audios para pruebas\000505440.wav,2.760000,fr
4,000513320.wav,D:\sentirobots\Audios para pruebas\000513320.wav,1.776009,fi
...,...,...,...,...
71,video44.wav,D:\sentirobots\Audios para pruebas\video44.wav,259.530159,None
72,video51.wav,D:\sentirobots\Audios para pruebas\video51.wav,146.913379,en
73,video60.wav,D:\sentirobots\Audios para pruebas\video60.wav,63.994875,en
74,video67.wav,D:\sentirobots\Audios para pruebas\video67.wav,61.649660,en


In [5]:
df_idioma.to_excel(r'D:\sentirobots\CSV de carpeta audios para pruebas\prueba-trans-idioma-detectado.xlsx', index=False)
print("idiomas detectados realizado neeneneeeeeeeeee")

idiomas detectados realizado neeneneeeeeeeeee
